In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
def jac_to_ic(rg,rp,theta,m1,m2,m3):
    
    thrad=np.radians(theta)

    mred2 = m2/(m2+m3)
    mred3 = m3/(m2+m3)

    r12=np.sqrt(rg*rg + mred3*mred3*rp*rp + 2.0*rg*mred3*rp*np.cos(thrad))
    r13=np.sqrt(rg*rg + mred2*mred2*rp*rp - 2.0*rg*mred2*rp*np.cos(thrad))
    r23=rp

    return r12,r13,r23

def ic_to_jac(r12,r13,r23,m1,m2,m3):

    mred2 = m2/(m1+m2)
    d1 = mred2*r13
    cbeta = (r12*r12+r13*r13-r23*r23)/(2.0*r12*r13)

    rp = r13
    rg = np.sqrt(r12*r12+d1*d1-2.0*r12*d1*cbeta)
    ctheta = (r12*r12-d1*d1-rg*rg)/(-2.0*rg*d1)
    theta = np.degrees(np.arccos(ctheta))

    return rg,rp,theta

def transform_coords(rg,rp,theta,m1,m2,m3):

    r12,r13,r23=jac_to_ic(rg,rp,theta,m1,m2,m3)
    rgp,rpp,thetap=ic_to_jac(r12,r13,r23,m1,m2,m3)

    return rgp,rpp,thetap

In [18]:
print(transform_coords(2.65,1.4,90,30.9737620,2.0141017778,2.0141017778))

(np.float64(2.5966922514026716), np.float64(2.7408940147331493), np.float64(148.58264539291366))


In [19]:
path_ex = "/home/jorgebdelafuente/Doctorado/Quantics/PH2p_1TApp/exact_test/"
rpgrid = np.loadtxt(path_ex+"rpgrid.dat",skiprows=11)[:,1]
rggrid = np.loadtxt(path_ex+"rggrid.dat",skiprows=11)[:,1]
thgrid = np.loadtxt(path_ex+"thgrid.dat",skiprows=11)[:,1]
print(rpgrid)

[0.5        0.52349869 0.54699739 0.57049608 0.59399478 0.61749347
 0.64099217 0.66449086 0.68798956 0.71148825 0.73498695 0.75848564
 0.78198433 0.80548303 0.82898172 0.85248042 0.87597911 0.89947781
 0.9229765  0.9464752  0.96997389 0.99347258 1.01697128 1.04046997
 1.06396867 1.08746736 1.11096606 1.13446475 1.15796345 1.18146214
 1.20496084 1.22845953 1.25195822 1.27545692 1.29895561 1.32245431
 1.345953   1.3694517  1.39295039 1.41644909 1.43994778 1.46344648
 1.48694517 1.51044386 1.53394256 1.55744125 1.58093995 1.60443864
 1.62793734 1.65143603 1.67493473 1.69843342 1.72193211 1.74543081
 1.7689295  1.7924282  1.81592689 1.83942559 1.86292428 1.88642298
 1.90992167 1.93342037 1.95691906 1.98041775 2.00391645 2.02741514
 2.05091384 2.07441253 2.09791123 2.12140992 2.14490862 2.16840731
 2.19190601 2.2154047  2.23890339 2.26240209 2.28590078 2.30939948
 2.33289817 2.35639687 2.37989556 2.40339426 2.42689295 2.45039164
 2.47389034 2.49738903 2.52088773 2.54438642 2.56788512 2.5913

In [ ]:
maxgrid = rggrid.shape[0]*rpgrid.shape[0]*thgrid.shape[0]

new_grid = np.zeros((maxgrid,3))

idx=0
for ith,th in enumerate(thgrid):
    for ig,rg in enumerate(rggrid):
        for ip,rp in enumerate(rpgrid):
            idx = ip + (ig)*rpgrid.shape[0] + (ith)*rpgrid.shape[0]*rggrid.shape[0]
            print(idx,ip,ig,ith)
            rgp,rpp,thetap = transform_coords(rg,rp,th,30.9737620,2.0141017778,2.0141017778)
            new_grid[idx,0], new_grid[idx,1], new_grid[idx,2] = rpp,rgp,thetap

rppmin = new_grid[:,0].min()
rppmax = new_grid[:,0].max()
rgpmin = new_grid[:,1].min()
rgpmax = new_grid[:,1].max()
thetamin = new_grid[:,2].min()
thetamax = new_grid[:,2].max()
print(rppmin,rppmax,rgpmin,rgpmax,thetamin,thetamax)


0 0 0 0
1 1 0 0
2 2 0 0
3 3 0 0
4 4 0 0
5 5 0 0
6 6 0 0
7 7 0 0
8 8 0 0
9 9 0 0
10 10 0 0
11 11 0 0
12 12 0 0
13 13 0 0
14 14 0 0
15 15 0 0
16 16 0 0
17 17 0 0
18 18 0 0
19 19 0 0
20 20 0 0
21 21 0 0
22 22 0 0
23 23 0 0
24 24 0 0
25 25 0 0
26 26 0 0
27 27 0 0
28 28 0 0
29 29 0 0
30 30 0 0
31 31 0 0
32 32 0 0
33 33 0 0
34 34 0 0
35 35 0 0
36 36 0 0
37 37 0 0
38 38 0 0
39 39 0 0
40 40 0 0
41 41 0 0
42 42 0 0
43 43 0 0
44 44 0 0
45 45 0 0
46 46 0 0
47 47 0 0
48 48 0 0
49 49 0 0
50 50 0 0
51 51 0 0
52 52 0 0
53 53 0 0
54 54 0 0
55 55 0 0
56 56 0 0
57 57 0 0
58 58 0 0
59 59 0 0
60 60 0 0
61 61 0 0
62 62 0 0
63 63 0 0
64 64 0 0
65 65 0 0
66 66 0 0
67 67 0 0
68 68 0 0
69 69 0 0
70 70 0 0
71 71 0 0
72 72 0 0
73 73 0 0
74 74 0 0
75 75 0 0
76 76 0 0
77 77 0 0
78 78 0 0
79 79 0 0
80 80 0 0
81 81 0 0
82 82 0 0
83 83 0 0
84 84 0 0
85 85 0 0
86 86 0 0
87 87 0 0
88 88 0 0
89 89 0 0
90 90 0 0
91 91 0 0
92 92 0 0
93 93 0 0
94 94 0 0
95 95 0 0
96 96 0 0
97 97 0 0
98 98 0 0
99 99 0 0
100 100 0 0
101 101 